In [1]:
import json
import time
import pandas as pd

from pyensign.ensign import authenticate, publisher, subscriber

# Configuration

In [2]:
ENSIGN_CREDS_PATH="ensign-creds.json"

# Subscribe to Flight Updates

In [3]:
callsigns = {}
training_list = []

In [4]:
def humanize(flight):
    callsign = flight['callsign']
    origin_country = flight['origin_country']
    heading = flight['true_track']
    velocity = flight['velocity']
    
    if callsign not in callsigns:
        callsigns[callsign] = flight
        return "Detected a new flight with callsign {} from {}, heading {} at {} m/s".format(callsign, origin_country, heading, velocity)
    else:
        return "{} is in progress at heading {} with velocity {} m/s".format(callsign, heading, velocity)

In [5]:
def store_data(flight):
    
    icao24 = flight['icao24']
    callsign = flight['callsign']
    origin_country = flight['origin_country']
    time_position = flight['time_position']
    last_contact = flight['last_contact']
    longitude = flight['longitude']
    latitude = flight['latitude']
    geo_altitude = flight['geo_altitude']
    on_ground = flight['on_ground']
    true_track = flight['true_track']
    velocity = flight['velocity']
    vertical_rate = flight['vertical_rate']
    sensors = flight['sensors']
    barometric_altitude = flight['barometric_altitude']
    transponder_code = flight['transponder_code']
    special_purpose_indicator = flight['special_purpose_indicator']
    position_source = flight['position_source']
    category = flight['category']

    return [
        icao24,
        callsign,
        origin_country,
        time_position,
        last_contact,
        longitude,
        latitude,
        geo_altitude,
        on_ground,
        true_track,
        velocity,
        vertical_rate,
        sensors,
        barometric_altitude,
        transponder_code,
        special_purpose_indicator,
        position_source,
        category
    ]    

        

In [6]:
@authenticate(cred_path=ENSIGN_CREDS_PATH)
@subscriber("flight-positions")
async def get_updates(updates):
    async for flight in updates:
        await print_update(flight)
   
@publisher("text-updates")
async def print_update(flight_event):
    flight = json.loads(flight_event.data)
    text = humanize(flight)

    # capturing data locally
    flight_data = store_data(flight)
    training_list.append(flight_data)
    
    return text

In [7]:
await get_updates()


CancelledError: 

In [9]:
df = pd.DataFrame(training_list)
df.to_csv("training_data.csv")